# به نام خدا

# فاز 1

In [6]:
import cv2
import os
import xml.etree.ElementTree as ET
from tqdm import tqdm

In [9]:


def parse_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    plates = []
    for obj in root.findall('object'):
        name = obj.find('name').text
        if name.lower() == 'vehicle plate':
            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            plates.append((xmin, ymin, xmax, ymax))
    
    return plates

def extract_plates(image_path, annotation_path, output_path):
    # خواندن تصویر
    image = cv2.imread(image_path)
    if image is None:
        print(f"خطا در خواندن تصویر: {image_path}")
        return
    
    # خواندن آنوتیشن‌ها
    plates = parse_annotation(annotation_path)
    if not plates:
        print(f"هیچ پلاکی در فایل آنوتیشن یافت نشد: {annotation_path}")
        return
    
    # استخراج و ذخیره هر پلاک
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    for i, (xmin, ymin, xmax, ymax) in enumerate(plates):
        plate_img = image[ymin:ymax, xmin:xmax]
        
        # ذخیره پلاک استخراج شده
        output_file = os.path.join(output_path, f"{base_name}.png")
        cv2.imwrite(output_file, plate_img)

def process_dataset(base_dir, output_dir):
    # مسیرهای ورودی
    images_dir = os.path.join(base_dir, "Vehicle Plates", "Vehicle Plates")
    annotations_dir = os.path.join(base_dir, "Vehicle Plates annotations", "Vehicle Plates annotations")
    
    # بررسی وجود پوشه‌ها
    if not os.path.exists(images_dir):
        print(f"پوشه تصاویر یافت نشد: {images_dir}")
        return
    if not os.path.exists(annotations_dir):
        print(f"پوشه آنوتیشن‌ها یافت نشد: {annotations_dir}")
        return
    
    # لیست تمام فایل‌های تصویر
    image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    # ایجاد پوشه خروجی اگر وجود نداشته باشد
    os.makedirs(output_dir, exist_ok=True)
    
    # پردازش هر تصویر با نوار پیشرفت
    for img_file in tqdm(image_files, desc="پردازش تصاویر"):
        base_name = os.path.splitext(img_file)[0]
        image_path = os.path.join(images_dir, img_file)
        annotation_path = os.path.join(annotations_dir, f"{base_name}.xml")
        
        if os.path.exists(annotation_path):
            extract_plates(image_path, annotation_path, output_dir)
        else:
            print(f"فایل آنوتیشن یافت نشد: {annotation_path}")

if __name__ == "__main__":
    # مسیرهای ورودی و خروجی
    base_dir = "Plates2"
    output_dir = "output_phase1"
    
    # اجرای پردازش
    process_dataset(base_dir, output_dir)
    
    print(f"پردازش فاز اول با موفقیت به پایان رسید. پلاک‌های استخراج شده در پوشه '{output_dir}' ذخیره شدند.")

پردازش تصاویر: 100%|██████████| 217/217 [00:00<00:00, 520.31it/s]

پردازش فاز اول با موفقیت به پایان رسید. پلاک‌های استخراج شده در پوشه 'output_phase1' ذخیره شدند.
